# Как пишут сети в 2к18

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
%matplotlib inline

In [ ]:
def get_loader(train, batch_size):
    # это скачает мнист и сохранит где-то рядом
    dataset = datasets.MNIST('mnist', train=train, download=True,
        transform=transforms.ToTensor())
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader
    
train = get_loader(True, 64)
val = get_loader(False, 64)

# Классификатор

Кроссэнтропия не очень информативна — она меряется в каких-то попугаях. Нас скорее интересует абсолютная точность классификации:

In [ ]:
def accuracy(model, val):
    total = 0
    correct = 0
    for X, y in val:
        X = X.view(-1, 784)
        res = model(X)
        res = res.argmax(dim=1)
        total += res.shape[0]
        correct += (res == y).sum().item()
    return correct / total

In [ ]:
model = nn.Sequential(
    nn.Linear(784, ???),
    # ...
    nn.Linear(???, 10),
    nn.Softmax(dim=1)
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = torch.nn.NLLLoss()
# ^ попробуйте какой-нибудь другой, если ещё не уверовали в кроссэнтропию

In [ ]:
train_losses = []
for epoch in range(20):
    for X, y in train:
        X = X.view(-1, 784)
        optimizer.zero_grad()
        
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        
        train_losses.append(loss.item())
        # как думаете, зачем нужен .item()?
        # подсказка: лосс хранит информацию о своей истории
        # попробуйте убрать .item() и посмотреть на расход памяти
        
        optimizer.step()
    
    print(accuracy(model, train), accuracy(model, val))
        
    plt.plot(train_losses)
    plt.show()

# Автоэнкодер

Давайте здесь уже напишем небольшой класс.

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encode = nn.Sequential(
            # ...
        )
        
        self.decode = nn.Sequential(
            # ...
            nn.Sigmoid()
            # картинки -- это тензоры со значениями от 0 до 1
            # нет особого смысла выводить что-то не из этого промежутка
        )
    
    def forward(self, x):
        return self.decode(self.encode(x))

model = Autoencoder()
criterial = torch.nn.MSELoss()
#                    ^ попробуйте также абсолютную ошибку
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
for epoch in range(10):
    train_loss = 0
    for data, _ in train:
        #     ^ лэйблы нам не нужны
        optimizer.zero_grad()
        
        reconstructed = model(data)
        loss = criterial(data, reconstructed)
        
        loss.backward()

        train_loss += loss.item()
        optimizer.step()

    print('epoch %d, loss %.4f' % (epoch, train_loss / len(train)))

Анимации `matplotlib` — это жесть. Не разбирайтесь в коде внизу, он просто нужен для плавных визуализаций.

In [ ]:
dataset = datasets.MNIST('mnist', train=train, download=True,
        transform=transforms.ToTensor())

def get(x):
    return dataset[x][0].view(1, 1, 28, 28)

def imshow(img):
    pic = img.numpy().astype('float')
    plt.axis('off')
    return plt.imshow(pic, cmap='Greys', animated=True)

def morph(inputs, steps, delay):
    latent = [model.encode(get(k)).data for k in inputs]
    fig = plt.figure()
    images = []
    for a, b in zip(latent, latent[1:] + [latent[0]]):
        for t in np.linspace(0, 1, steps):
            c = a*(1-t)+b*t
            morphed = model.decode(c).data
            morphed = morphed.view(28, 28)
            images.append([imshow(morphed)])
    
    ani = animation.ArtistAnimation(fig, images, interval=delay)

    display(HTML(ani.to_html5_video()))

In [ ]:
morph(np.random.randint(0, len(dataset), 30), 20, 30)